# 门控制循环单元GRU
- rnn中由于时间步数过大时，梯度中的连乘项引起了梯度的衰减和爆炸。小数或大于1的数，连乘会导致衰减和爆炸
- 门控制单元，目的是更好捕捉时间步距离长时的梯度，避免衰减。是一种常用的门控循环神经网络。

## 门控制循环单元的结构

- 门控单元目标：不让梯度消失
- 门控单元的做法：将会导致梯度消失的隐藏状态丢掉，加大对当前层的隐藏状态的保存
- 结构上的设计
   1. R门：重置门
   2. Z门：控制更新门

![GRU图示](./img/GRU.PNG)

- 为何需要长记忆呢？因为有的因果推断和前面很远的一句话有关系：

![为何使用长记忆](./img/whyLongMem.gif)


问题：只保留了较远端的信息，那较近部分的信息都没了，整个网络的预测会不会很不准确

## 读取数据集

In [1]:
import d2lzh as d2l
from mxnet import nd
from mxnet.gluon import rnn

In [2]:
(corpus_indices, char_to_idx, idx_to_char, 
vocab_size) = d2l.load_data_jay_lyrics()

## 初始化模型参数 

In [3]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()


def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                nd.zeros(num_hiddens, ctx=ctx)
                )

    W_xz, W_hz, b_z = _three()  # 更新⻔参数
    W_xr, W_hr, b_r = _three()  # 重置⻔参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 附上梯度
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

## 定义模型

In [4]:
def init_gru_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [5]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = nd.sigmoid(nd.dot(X, W_xz) + nd.dot(H, W_hz) + b_z)
        R = nd.sigmoid(nd.dot(X, W_xr) + nd.dot(H, W_hr) + b_r)
        H_tilda = nd.tanh(nd.dot(X, W_xh) + nd.dot(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)


## 训练并创作歌词

In [6]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [7]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, ctx, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 154.603487, time 6.93 sec
 - 分开 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你
 - 不分开 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你 我想你你
epoch 80, perplexity 34.885169, time 8.12 sec
 - 分开 我想要你的微笑 一直后觉 我想要这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 
 - 不分开 我想要你的微笑 一直后觉 我想要你的微笑 我想要你 你是我 想你的话笑不知 我想 你不很久 我不要
epoch 120, perplexity 6.051254, time 7.89 sec
 - 分开 我想要你的微笑每天 想要和你 其实我有起经 有话去对医药箱说 别怪我 别怪我 说你怎么么对我 甩开
 - 不分开 我已道这样奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 静静后觉 后知了这样的
epoch 160, perplexity 1.841687, time 7.28 sec
 - 分开 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤哀 我 想带你骑单车 我 想和你看棒球 想这样
 - 不分开 我一定会护护你 也逗我进做汉着 我想要这样牵着你的手不放开 爱能不能够永远单纯没有悲哀 我 想带你
